# Multimode gmm
Created by: Daniel L. Marino (marinodl@vcu.edu)

Description: gaussian mixture model implementation on tensorflow, the objective is a multimode gaussian distribution. 

This file serves as a visual test for the gmm extension modules

In [1]:
import numpy as np
import collections
from time import time
import tensorflow as tf
from twodlearn.tf_lib.GMM import *
from twodlearn.tf_lib.datasets.generic import Datasets

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 1. Create dataset

In [2]:

def gmm_sampling(mu, sigma, w, n_samples=1):
    # generates random vectors, sampled from a gausian mixture model
    #  
    #     - mu: 3d tensor containing the means for the gaussians.
    #           the "depth" dimention (3rd) is used to index the
    #           gaussians.    [ kernel_id, dim]
    #     - sigma: 3d tensor containing the covariance matrix of the
    #              gaussians. [ kernel_id, dim] for diagonal matrices
    #     - w: vector in form of a 3d tensor containing the weights
    #          for each one of the gaussians, they have to sum one. 
    #          [kernel_id]
    n_kernels = mu.shape[0]
    n_dim = mu.shape[1]
    
    # random sample the kernel from which the output samples are going
    # to be drawn
    kernel_id= np.argmax(np.random.multinomial(1, w, size=[n_samples]), axis=1 )
    
    out = np.zeros([n_samples, n_dim])
    for i in range(n_samples):
        out[i,:]= np.random.multivariate_normal(mu[kernel_id[i],:], np.diag(sigma[kernel_id[i],:])) # if diagonal
    
    return out;
    
    

In [3]:
n_samples = 5000000

n_kernels_r = 2
n_dim = 2
mu_r = np.array([[1, 1], [10, 10]])
sigma_r = np.array([[0.5, 0.5], [2, 2]])
w_r = [1/n_kernels_r]*n_kernels_r # has to sum to one

# random sample from reference distribution
aux_x = gmm_sampling(mu_r, sigma_r, w_r, n_samples);

# build the dataset
data = Datasets(aux_x)
data.normalize()
print('Data shape: ', data.train.x.shape)

# plot
# plt.plot(data.train.x[:,0], data.train.x[:,1], 'o')

Data shape:  (5000000, 2)


## 2. Model definition

In [4]:
#config = tf.ConfigProto( device_count = {'GPU': 0} )
#sess = tf.InteractiveSession(config = config)
sess = tf.InteractiveSession()

In [5]:
n_kernels = 2

gmm_model = GmmShallowModel( n_dim, n_kernels, diagonal= True, method='tdl')

train_model= gmm_model.setup(n_samples)

In [6]:
# Optimizer.
optimizer = tf.train.AdamOptimizer(0.05).minimize(train_model.loss) #0.001

In [7]:
print('mu shape:', train_model.mu.get_shape())
print('sigma shape:', train_model.sigma.get_shape())
print('w shape:', train_model.w.get_shape())

print('out shape:', train_model.out.get_shape())

print('loss shape:', train_model.loss.get_shape())


mu shape: (1, 2, 2)
sigma shape: (1, 2, 2)
w shape: (1, 2)
out shape: <unknown>
loss shape: <unknown>


## 3. Training

In [8]:
num_steps = 1000 #1000
n_logging = 100
n_test_logg = 10

tf.initialize_all_variables().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0

t0 = time()
for step in range(num_steps):   
    
    _, l = sess.run([optimizer, train_model.loss],feed_dict={train_model.inputs : data.train.x})
    mean_loss += l    
    
    
    if step%n_logging == 0:                
        print(step, ' | loss:', mean_loss/n_logging)
        mean_loss= 0

        
t1 = time()
print('function takes: ', (t1-t0))

Initialized
0  | loss: 141893.93
100  | loss: 5743661.01
200  | loss: 2316218.5475
300  | loss: 2315581.655
400  | loss: 2315581.7375
500  | loss: 2315582.0325
600  | loss: 2315581.9975
700  | loss: 2315581.955
800  | loss: 2315581.9725
900  | loss: 2315581.94
function takes:  65.62872743606567


## 4. Plot

In [9]:
n_plot= [20, 20];

# model for testing
n_test= n_plot[0]*n_plot[1];
test_model = gmm_model.setup(n_test)


In [10]:

xv, yv = np.meshgrid(np.linspace(-2, 2, n_plot[1]), np.linspace(-2, 2, n_plot[0]))

x_plot = np.stack([xv.transpose(), yv.transpose()], 2)

x_plot = np.reshape(x_plot, [-1,2])

[p_plot, mu_out, sigma_out, w_out,] = sess.run([test_model.out, test_model.mu, test_model.sigma, test_model.w],
                                               feed_dict= {test_model.inputs : x_plot})
print('mu:', mu_out, 'sigma:', sigma_out, 'w:', w_out)


p_plot = np.reshape(p_plot, [n_plot[1], n_plot[0]])
print(p_plot.shape)

# plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_wireframe(xv, yv, p_plot)



mu: [[[-0.97018242 -0.97014004]
  [ 0.97085959  0.97081721]]] sigma: [[[ 0.02327428  0.02328142]
  [ 0.09292822  0.09308528]]] w: [[ 0.50090683  0.4990932 ]]
(20, 20)


<IPython.core.display.Javascript object>